In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [ ]:
%%writefile XLMROBERTA_TPU.py

import argparse
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
#from apex import amp
import random
import re
import json
from transformers import ( 
    BertTokenizer, 
    AdamW, 
    BertModel, 
    BertForPreTraining,
    BertConfig,
    get_linear_schedule_with_warmup,
    BertTokenizerFast,
    RobertaModel,
    RobertaTokenizerFast,
    RobertaConfig,
    AlbertTokenizer,
    AlbertConfig,
    AlbertModel
)
import transformers

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

import sentencepiece as spm
#import sentencepiece_pb2

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(0, "../input/sentencepiece-pb2/")
import sentencepiece_pb2


def to_list(tensor):
    return tensor.detach().cpu().tolist()

class AverageMeter(object):
    """Computes and stores the average and current values"""
    def __init__(self):
        self.reset()
    
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def get_position_accuracy(logits, labels):
    predictions = np.argmax(F.softmax(logits, dim=1).cpu().data.numpy(), axis=1)
    labels = labels.cpu().data.numpy()
    total_num = 0
    sum_correct = 0
    for i in range(len(labels)):
        if labels[i] >= 0:
            total_num += 1
            if predictions[i] == labels[i]:
                sum_correct += 1
    if total_num == 0:
        total_num = 1e-7
    return np.float32(sum_correct) / total_num, total_num

def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / ((len(a) + len(b)) - len(c))

def calculate_jaccard_score(features_dict, start_logits, end_logits, tokenizer):

    input_ids = to_list(features_dict["input_ids"])
    #start_position = to_list(features_dict["start_position"])
    #end_position = to_list(features_dict["end_position"])
    tweet = features_dict["tweet"]
    selected_text = features_dict["selected_text"]
    sentiment = features_dict["sentiment"]
    #offsets = features_dict["offsets"]

    start_logits = np.argmax(F.softmax(start_logits, dim=1).cpu().data.numpy(), axis=1)
    end_logits = np.argmax(F.softmax(end_logits, dim=1).cpu().data.numpy(), axis=1)

    jac_list = []

    for i in range(len(tweet)):

        idx_start = start_logits[i]
        idx_end = end_logits[i]
        #offset = offsets[i]
        input_id = input_ids[i]
        tw = tweet[i]
        target_st = selected_text[i]

        if idx_end < idx_start:
            idx_end = idx_start

        """
        filtered_output = ""
        for ix in range(idx_start, idx_end):
            filtered_output += tw[offset[ix][0]: offset[ix][1]]
            if (ix+1) < len(offset) and offset[ix][1] < offset[ix+1][0]:
                filtered_output += " "
        """
        filtered_output = tokenizer.decode(input_id[idx_start:idx_end+1], skip_special_tokens=True)

        if sentiment[i] == "neutral" or len(tw.split()) < 2:
            filtered_output = tw
        
        
        jac = jaccard(target_st.strip(), filtered_output.strip())

        jac_list.append(jac)

    return np.mean(jac_list)

########## Datapreparation & Dataset ###########

def find_start_and_end(tweet, selected_text):
    len_st = len(selected_text)
    start = None
    end = None
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind+len_st] == selected_text:
            start = ind
            end = ind + len_st - 1
            break
    return start, end

def _is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

def whitespace_tokenizer(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    #print(text)
    if not text:
         return []
    tokens = text.split()
    return tokens

def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer, orig_answer_text):
    """Returns tokenized answer spans that better match the annotated answer."""
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))
     
    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start : (new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)
    return (input_start, input_end)

class SentencePieceTokenizer:
    def __init__(self, model_path):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(os.path.join(model_path))
    
    def encode(self, sentence):
        spt = sentencepiece_pb2.SentencePieceText()
        spt.ParseFromString(self.sp.encode_as_serialized_proto(sentence))
        offsets = []
        tokens = []
        for piece in spt.pieces:
            tokens.append(piece.id)
            offsets.append((piece.begin, piece.end))
        return tokens, offsets

def find_start_end_offsets(tweet, selected_text, start_position_character, tokenizer):
    
    start_position = 0
    end_position = 0
    
    tweet_tokens = []
    
    char_to_word_offset = []
    prev_is_whitespace = True
    
    # Split on whitespace so that different tokens may be attributed to their original position.
    for c in tweet:
        if _is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                tweet_tokens.append(c)
            else:
                tweet_tokens[-1] += c
            prev_is_whitespace = False
        char_to_word_offset.append(len(tweet_tokens) - 1)
    
    # Start and end positons only has a value during evalution.
    if start_position_character is not None:
        start_position = char_to_word_offset[start_position_character]
        end_position = char_to_word_offset[
                min(start_position_character + len(selected_text) - 1, len(char_to_word_offset) -1)                                  
        ]
        
    tok_to_orig_index = []
    orig_to_tok_index = []
    all_tweet_tokens = []
    
    for (i, token) in enumerate(tweet_tokens):
        orig_to_tok_index.append(len(all_tweet_tokens))
        sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
            tok_to_orig_index.append(i)
            all_tweet_tokens.append(sub_token)
    #print(orig_to_tok_index)
    tok_start_position = orig_to_tok_index[start_position]
    if end_position < len(tweet_tokens) - 1:
        tok_end_position = orig_to_tok_index[end_position + 1] - 1
    else:
        tok_end_position = len(all_tweet_tokens) - 1
    
     
    
    (tok_start_position, tok_end_position) = _improve_answer_span(
            all_tweet_tokens, tok_start_position, tok_end_position, tokenizer, selected_text
        )
    
    return tok_start_position, tok_end_position

def process_with_offsets(args, tweet, selected_text, sentiment, tokenizer, spt):

    start_index, end_index = find_start_and_end(tweet, selected_text)

    char_targets = [0]*len(tweet)
    if start_index != None and end_index != None:
        for ct in range(start_index, end_index+1):
            char_targets[ct] = 1
    
    encoded = tokenizer.encode_plus(
                    sentiment,
                    tweet,
                    max_length=args.max_seq_len,
                    pad_to_max_length=True,
                    return_token_type_ids=True,
                    #return_offsets_mapping=True
                )
    
    len_sentences_pair_tokens = tokenizer.max_len - tokenizer.max_len_sentences_pair + 2

    offsets = spt.encode(tweet)[-1]
    encoded["offset_mapping"] = [(0,0)]*4 + offsets

    target_idx = []
    for j, (offset1, offset2) in enumerate(encoded["offset_mapping"]):
        if j > 3:#(len_sentences_pair_tokens + 2) - 1 :
            if sum(char_targets[offset1:offset2]) > 0:
                target_idx.append(j)

    #sp , ep = find_start_end_offsets(tweet, selected_text, start_index, tokenizer)
    #print(sp, ep)
    
    
    encoded["start_position"] = target_idx[0]
    encoded["end_position"] = target_idx[-1]
    encoded["tweet"] = tweet
    encoded["selected_text"] = selected_text
    encoded["sentiment"] = sentiment

    return encoded

class TweetDataset:
    def __init__(self, args, tokenizer, spt , df, mode="train", fold=0):
        
        self.mode = mode

        if self.mode == "train":
            df = df[~df.kfold.isin([fold])].dropna()
            self.tweet = df.text.values
            self.sentiment = df.sentiment.values
            self.selected_text = df.selected_text.values
        
        elif self.mode == "valid":
            df = df[df.kfold.isin([fold])].dropna()
            self.tweet = df.text.values
            self.sentiment = df.sentiment.values
            self.selected_text = df.selected_text.values
        
        self.tokenizer = tokenizer
        self.args = args
        self.spt = spt
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):

        tweet = str(self.tweet[item])
        selected_text = str(self.selected_text[item])
        sentiment = str(self.sentiment[item])
        
        features = process_with_offsets(
                        args=self.args, 
                        tweet=tweet, 
                        selected_text=selected_text, 
                        sentiment=sentiment, 
                        tokenizer=self.tokenizer,
                        spt=self.spt
                    )
        
        return {
            "input_ids":torch.tensor(features["input_ids"], dtype=torch.long),
            "token_type_ids":torch.tensor(features["token_type_ids"], dtype=torch.long),
            "attention_mask":torch.tensor(features["attention_mask"], dtype=torch.long),
            #"offsets":features["offset_mapping"],
            "start_position":torch.tensor(features["start_position"],dtype=torch.long),
            "end_position":torch.tensor(features["end_position"], dtype=torch.long),

            "tweet":features["tweet"],
            "selected_text":features["selected_text"],
            "sentiment":features["sentiment"]

        }

class BertForQuestionAnswering(BertForPreTraining):
    """
    BERT model for QA

    Parameters
    ----------
    config : transformers.BertConfig. Configuration class for BERT.

    Returns
    -------
    start_logits : torch.Tensor with shape (batch_size, sequence_size)
        Starting scores of each tokens.
    end_logits : torch.Tenosr with shape (batch_size, sequence_size).
        Ending scores of each tokens.
    """

    def __init__(self, config):
        super(BertForQuestionAnswering, self).__init__(config)
        self.bert = RobertaModel(config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            #token_type_ids=token_type_ids,
                            #position_ids=position_ids,
                            #head_mask=head_mask
                        )
        sequence_output = outputs[0]
        #pooled_output = outputs[1]

        # predict start & end position
        qa_logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, model_path, conf):
        super(TweetModel, self).__init__(conf)
        self.xlmroberta = transformers.XLMRobertaModel.from_pretrained(model_path, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2) #768
        torch.nn.init.normal_(self.l0.weight, std=0.02)


    def forward(self,input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        _,_, out = self.xlmroberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        #print(out.shape)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

def reduce_fn(vals):
    return sum(vals) / len(vals)

def loss_fn(preds, labels):
    start_preds, end_preds = preds
    start_labels, end_labels = labels

    start_loss = nn.CrossEntropyLoss(ignore_index=-1)(start_preds, start_labels)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_preds, end_labels)
    return start_loss, end_loss

def train(args, train_loader, model, optimizer,scheduler, epoch, f):
    total_loss = AverageMeter()
    losses1 = AverageMeter() # start
    losses2 = AverageMeter() # end
    accuracies1 = AverageMeter() # start
    accuracies2 = AverageMeter() # end

    model.train()

    t = tqdm(train_loader, disable=not xm.is_master_ordinal())
    for step, d in enumerate(t):
        
        input_ids = d["input_ids"].to(args.device)
        attention_mask = d["attention_mask"].to(args.device)
        token_type_ids = d["token_type_ids"].to(args.device)
        start_position = d["start_position"].to(args.device)
        end_position = d["end_position"].to(args.device)

        model.zero_grad()

        logits1, logits2 = model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids, 
            position_ids=None, 
            head_mask=None
        )

        y_true = (start_position, end_position)
        loss1, loss2 = loss_fn((logits1, logits2), (start_position, end_position))
        loss = loss1 + loss2

        acc1, n_position1 = get_position_accuracy(logits1, start_position)
        acc2, n_position2 = get_position_accuracy(logits2, end_position)

        total_loss.update(loss.item(), n_position1)
        losses1.update(loss1.item(), n_position1)
        losses2.update(loss2.item(), n_position2)
        accuracies1.update(acc1, n_position1)
        accuracies2.update(acc2, n_position2)

        
        #optimizer.zero_grad()
        #with amp.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        loss.backward()
        xm.optimizer_step(optimizer)
        scheduler.step()
        print_loss = xm.mesh_reduce("loss_reduce", total_loss.avg, reduce_fn)
        print_acc1 = xm.mesh_reduce("acc1_reduce", accuracies1.avg, reduce_fn)
        print_acc2 = xm.mesh_reduce("acc2_reduce", accuracies2.avg, reduce_fn)
        t.set_description(f"Train E:{epoch+1} - Loss:{print_loss:0.2f} - acc1:{print_acc1:0.2f} - acc2:{print_acc2:0.2f}")


    log_ = f"Epoch : {epoch+1} - train_loss : {total_loss.avg} - \n \
    train_loss1 : {losses1.avg} - train_loss2 : {losses2.avg} - \n \
    train_acc1 : {accuracies1.avg} - train_acc2 : {accuracies2.avg}"

    f.write(log_ + "\n\n")
    f.flush()
    
    return total_loss.avg

def valid(args, valid_loader, model, tokenizer, epoch, f):
    total_loss = AverageMeter()
    losses1 = AverageMeter() # start
    losses2 = AverageMeter() # end
    accuracies1 = AverageMeter() # start
    accuracies2 = AverageMeter() # end

    jaccard_scores = AverageMeter()

    model.eval()

    with torch.no_grad():
        t = tqdm(valid_loader, disable=not xm.is_master_ordinal())
        for step, d in enumerate(t):
            
            input_ids = d["input_ids"].to(args.device)
            attention_mask = d["attention_mask"].to(args.device)
            token_type_ids = d["token_type_ids"].to(args.device)
            start_position = d["start_position"].to(args.device)
            end_position = d["end_position"].to(args.device)

            logits1, logits2 = model(
                input_ids=input_ids, 
                attention_mask=attention_mask, 
                token_type_ids=None, 
                position_ids=None, 
                head_mask=None
            )

            y_true = (start_position, end_position)
            loss1, loss2 = loss_fn((logits1, logits2), (start_position, end_position))
            loss = loss1 + loss2

            acc1, n_position1 = get_position_accuracy(logits1, start_position)
            acc2, n_position2 = get_position_accuracy(logits2, end_position)

            total_loss.update(loss.item(), n_position1)
            losses1.update(loss1.item(), n_position1)
            losses2.update(loss2.item(), n_position2)
            accuracies1.update(acc1, n_position1)
            accuracies2.update(acc2, n_position2)

            jac_score = calculate_jaccard_score(features_dict=d, start_logits=logits1, end_logits=logits2, tokenizer=tokenizer)

            jaccard_scores.update(jac_score)

            print_loss = xm.mesh_reduce("vloss_reduce", total_loss.avg, reduce_fn)
            print_jac = xm.mesh_reduce("jac_reduce", jaccard_scores.avg, reduce_fn)

            t.set_description(f"Eval E:{epoch+1} - Loss:{print_loss:0.2f} - Jac:{print_jac:0.2f}")

    #print("Valid Jaccard Score : ", jaccard_scores.avg)
    log_ = f"Epoch : {epoch+1} - valid_loss : {total_loss.avg} - \n\
    valid_loss1 : {losses1.avg} - \valid_loss2 : {losses2.avg} - \n\
    valid_acc1 : {accuracies1.avg} - \valid_acc2 : {accuracies2.avg} "

    f.write(log_ + "\n\n")
    f.flush()
    
    return jaccard_scores.avg


def main():

    parser = argparse.ArgumentParser()

    parser.add_argument("--local_rank", type=int, default=-1, help="local_rank for distributed training on gpus")
    parser.add_argument("--max_seq_len", type=int, default=192)
    parser.add_argument("--fold_index", type=int, default=0)
    parser.add_argument("--learning_rate", type=float, default=0.00002)
    parser.add_argument("--epochs", type=int, default=5)
    parser.add_argument("--batch_size", type=int, default=16)
    parser.add_argument("--model_path", type=str, default="roberta-base")
    parser.add_argument("--output_dir", type=str, default="")
    parser.add_argument("--exp_name", type=str, default="")
    parser.add_argument("--spt_path", type=str, default="")

    args = parser.parse_args()

    # Setting seed
    seed = 42
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

     # hperparameters
    #args.max_seq_len = 192
    #args.learning_rate = 0.00002
    #args.batch_size = 16
    #args.epochs = 5
    #args.fold_index = 0

    #model_path = "../huggingface_pretrained/bert-base-uncased/"
    model_path = args.model_path
    config = transformers.XLMRobertaConfig.from_pretrained(model_path)
    config.output_hidden_states = True
    tokenizer = transformers.XLMRobertaTokenizer.from_pretrained(model_path, do_lower_case=True)
    #model = BertForQuestionAnswering.from_pretrained(model_path, config=config)
    MX = TweetModel(model_path, config)

    spt = SentencePieceTokenizer(args.spt_path)

    train_df = pd.read_csv("../input/tweet-create-folds/train_3folds.csv")
    #train_df = train_df[train_df.sentiment != "neutral"]
    print(len(train_df))

    args.save_path = os.path.join(args.output_dir, args.exp_name)

    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

    f = open(os.path.join(args.save_path, f"log_f_{args.fold_index}.txt"), "w")

    def run():

        args.device = xm.xla_device()
        model = MX.to(args.device)


        # DataLoaders
        train_dataset = TweetDataset(
            args=args,
            df=train_df,
            mode="train",
            fold=args.fold_index,
            tokenizer=tokenizer,
            spt=spt
        )
        train_sampler = torch.utils.data.distributed.DistributedSampler(
            train_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True
        )
        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=args.batch_size,
            sampler=train_sampler,
            drop_last=True,
            num_workers=2
        )


        valid_dataset = TweetDataset(
            args=args,
            df=train_df,
            mode="valid",
            fold=args.fold_index,
            tokenizer=tokenizer,
            spt=spt
        )
        valid_sampler = torch.utils.data.distributed.DistributedSampler(
            valid_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=args.batch_size,
            sampler=valid_sampler,
            num_workers=1,
            drop_last=False
        )

        #optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
        #model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

        num_train_steps = int(len(train_df) / args.batch_size * args.epochs)
    
        param_optimizer = list(model.named_parameters())
        no_decay = [
            "bias",
            "LayerNorm.bias",
            "LayerNorm.weight"
        ]
        optimizer_parameters = [
            {
                'params': [
                    p for n, p in param_optimizer if not any(
                        nd in n for nd in no_decay
                    )
                ], 
            'weight_decay': 0.001
            },
            {
                'params': [
                    p for n, p in param_optimizer if any(
                        nd in n for nd in no_decay
                    )
                ], 
                'weight_decay': 0.0
            },
        ]

        num_train_steps = int(
            len(train_df) / args.batch_size / xm.xrt_world_size() * args.epochs
        )

        optimizer = AdamW(
            optimizer_parameters,
            lr=args.learning_rate * xm.xrt_world_size()
        )
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=num_train_steps
        )

        xm.master_print("Training is Starting.....")
        best_jac = -1000

        for epoch in range(args.epochs):
            para_loader = pl.ParallelLoader(train_loader, [args.device])
            train_loss = train(
                args, 
                para_loader.per_device_loader(args.device),
                model, 
                optimizer,
                scheduler,
                epoch,
                f
            )
            para_loader = pl.ParallelLoader(valid_loader, [args.device])
            valid_jac = valid(
                args, 
                para_loader.per_device_loader(args.device),
                model, 
                tokenizer,
                epoch,
                f
            )

            jac = xm.mesh_reduce("jac_reduce", valid_jac, reduce_fn)
            xm.master_print(f"**** Epoch {epoch+1} **==>** Jaccard = {jac}")

            log_ = f"**** Epoch {epoch+1} **==>** Jaccard = {jac}"

            f.write(log_ + "\n\n")

            

            if jac > best_jac:
                xm.master_print("**** Model Improved !!!! Saving Model")
                xm.save(model.state_dict(), os.path.join(args.save_path, f"fold_{args.fold_index}"))
                best_jac = jac


    
    def _mp_fn(rank, flags):
        torch.set_default_tensor_type('torch.FloatTensor')
        a = run()
    
    FLAGS={}
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=1, start_method='fork')


if __name__ == "__main__":
    main()

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model

In [ ]:
!python XLMROBERTA_TPU.py --fold_index=0 \
                  --model_path="xlm-roberta-base" \
                  --spt_path="xlm-roberta-base-sentencepiece.bpe.model" \
                  --output_dir="xlm-roberta-base" \
                  --exp_name="xlm-roberta-base" \
                  --batch_size=64

In [ ]:
!python XLMROBERTA_TPU.py --fold_index=1 \
                  --model_path="xlm-roberta-base" \
                  --spt_path="albert-xlarge-v2-spiece.model" \
                  --output_dir="xlm-roberta-base" \
                  --exp_name="xlm-roberta-base" \
                  --batch_size=64

In [ ]:
!python XLMROBERTA_TPU.py --fold_index=2 \
                  --model_path="xlm-roberta-base" \
                  --spt_path="albert-xlarge-v2-spiece.model" \
                  --output_dir="xlm-roberta-base" \
                  --exp_name="xlm-roberta-base" \
                  --batch_size=64